<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 0 - Introducción a Redes Neuronales </H3>
<div align='center'>
###  Felipe González - 201273534-3 
### Ignacio Tampe - 201573514-k 
</div>
<hr style="height:2px;border:none"/>

## 1. Back-propagation (BP) from *Scratch*


In [18]:
import numpy as np #se importa numpy como np

In [19]:
class RedNeuronal:
    # num_entrada: dimension de los vectores de entrada, en iris: 4
    # tam_capas: lista de cantidad de neuronas por cada capa interna, en iris: [32, 16]
    # num_salida: dimension de los vectores de salida, en iris: 3
    def __init__(self, num_entrada, tam_capas, num_salida):
        self.num_entrada = num_entrada
        self.tam_capas = tam_capas
        self.num_salida = num_salida
        self.capas = []
        # iniciar todas las capas inicial - ocultas - final
        self.capas.append(CapaNeuronal(num_entrada))
        for i in range(1, len(tam_capas)):
            self.capas.append(CapaNeuronal(tam_capas[i-1], self.capas[i-1].tam_capa))
        self.capas.append(CapaNeuronal(num_salida, self.capas[len(self.capas)-1].tam_capa))
        
    def train(self, entrada, output, ciclos, tasa):
        # repetir para todas 
        for ciclo in range(ciclos):
            for vector in entrada:
                self.capas[0].set_capa(vector) #Poblar capa inicial
                # Feed forward
                for i in range(1, len(self.capas)):
                    self.capas[i].feed_forward(self.capas[i-1])
                
                self.capas[len(self.capas)-1].mostrar_capa()
                
                # evaluar error

                # Back propagation (actualizar pesos de las neuronas)
            
            



In [20]:
class CapaNeuronal:
    def __init__(self, tam_capa, tam_anterior = 0):
        self.tam_capa = tam_capa
        self.bias = 1
        self.neuronas = [Neurona(tam_anterior)] * tam_capa # lista de neuronas de la capa
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def derivate_sigmoid(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))
    
    
    
    def get_activaciones(self):
        return [neurona.activacion for neurona in self.neuronas]
    
    def feed_forward(self, capa_anterior):
        for neurona in self.neuronas:
            activaciones = capa_anterior.get_activaciones()
            pesos = neurona.pesos
            result = self.sigmoid(np.dot(pesos, activaciones) + self.bias)          
            neurona.set_activacion(result)
            
    #Funcion para poblar datos de entrada a la red.
    def set_capa(self, valores):
        for i in range(len(self.neuronas)):
            self.neuronas[i].set_activacion(valores[i])
            
    def mostrar_capa(self):
        for neurona in self.neuronas:
            print(neurona, end="\t")
        print("\n-----------------------------")

        
class Neurona:
    def __init__(self, tam_capa_anterior):
        self.pesos = np.random.rand(tam_capa_anterior) # genera una lista de numeros aleatorios de tamaño "tam_capa_anterior"
        
    def set_activacion(self, valor):
        self.activacion = valor
        
    def __str__(self):
        return "Activación: "+str(self.activacion)

### ¡Es hora de probar!

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
X_train,y_train = load_iris(return_X_y=True)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
#transform target to one hot vector
nn = RedNeuronal(4, [32, 16], 3)
nn.train(X_train, y_train, 1, 0.1)

Activación: 0.988328558682	Activación: 0.988328558682	Activación: 0.988328558682	
-----------------------------
Activación: 0.988328558682	Activación: 0.988328558682	Activación: 0.988328558682	
-----------------------------
Activación: 0.988328558682	Activación: 0.988328558682	Activación: 0.988328558682	
-----------------------------
Activación: 0.988328558682	Activación: 0.988328558682	Activación: 0.988328558682	
-----------------------------
Activación: 0.988328558682	Activación: 0.988328558682	Activación: 0.988328558682	
-----------------------------
Activación: 0.997402799338	Activación: 0.997402799338	Activación: 0.997402799338	
-----------------------------
Activación: 0.994221330411	Activación: 0.994221330411	Activación: 0.994221330411	
-----------------------------
Activación: 0.988328558682	Activación: 0.988328558682	Activación: 0.988328558682	
-----------------------------
Activación: 0.988328558682	Activación: 0.988328558682	Activación: 0.988328558682	
----------------------

## 1. Referencias


In [22]:
import keras
y_onehot = keras.utils.to_categorical(y_train)

In [23]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17290374944691376189
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 174057062
locality {
  bus_id: 1
}
incarnation: 17761320190815713119
physical_device_desc: "device: 0, name: GeForce 920M, pci bus id: 0000:04:00.0"
]


### Pregunta 2

In [24]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation="sigmoid"))
model.add(Dense(16, activation="sigmoid"))
model.add(Dense(3, activation="softmax"))
model.compile(optimizer=SGD(lr=0.1),loss="mse", metrics=["accuracy"])

In [25]:
model.fit(X_train, y_onehot, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
150/150 [==============================] - 1s 5ms/step - loss: 0.2142 - acc: 0.4667
Epoch 2/100
150/150 [==============================] - 1s 4ms/step - loss: 0.1933 - acc: 0.6133
Epoch 3/100
150/150 [==============================] - 1s 4ms/step - loss: 0.1596 - acc: 0.6733
Epoch 4/100
150/150 [==============================] - 1s 4ms/step - loss: 0.1331 - acc: 0.7467
Epoch 5/100
150/150 [==============================] - 1s 4ms/step - loss: 0.1135 - acc: 0.7933
Epoch 6/100
150/150 [==============================] - 1s 4ms/step - loss: 0.1004 - acc: 0.8333
Epoch 7/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0895 - acc: 0.8867
Epoch 8/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0793 - acc: 0.8933
Epoch 9/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0726 - acc: 0.9133
Epoch 10/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0642 - acc: 0.8933
Epoch 11/100
150/150 [=================

150/150 [==============================] - 1s 4ms/step - loss: 0.0107 - acc: 0.9800
Epoch 86/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0127 - acc: 0.9733
Epoch 87/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0111 - acc: 0.9800
Epoch 88/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0115 - acc: 0.9733
Epoch 89/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0120 - acc: 0.9733
Epoch 90/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0113 - acc: 0.9733
Epoch 91/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0100 - acc: 0.9867
Epoch 92/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0128 - acc: 0.9800
Epoch 93/100
150/150 [==============================] - 1s 5ms/step - loss: 0.0123 - acc: 0.9667
Epoch 94/100
150/150 [==============================] - 1s 4ms/step - loss: 0.0093 - acc: 0.9867
Epoch 95/100
150/150 [=====================